In [1]:
from ezjob import *


  ______ _____ _       _
 |  ____|___  (_)     | |
 | |__     / / _  ___ | |__   
 |  __|   / / | |/ _ \| '_ \    
 | |____ / /__| | (_) | |_) |     
 |______/_____| |\___/|_.__/     
             _/ |
            |__/
   
Job Manager version :    0.1.8
Today :    2020-07-22 03:46:33


In [3]:
#SSH takes 2 args <config_file_path/dict>, <name>
#<name> is the folder name to be created at local, this local folder will mirror to REMOTE_WORKING_DIR
#make sure your REMOTE_WORKING_DIR in config file exsit in remote
ssh=SSH('config_rcc.ini','rcc')
ssh.connect()

Password:
Duo two-factor login for huangl1

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4177
 2. Phone call to XXX-XXX-4177
 3. SMS passcodes to XXX-XXX-4177

Passcode or option (1-3):
Success. Logging you in...
Last login: Wed Jul 22 03:39:49 2020 from c-73-51-208-105.hsd1.il.comcast.net
                               Welcome to Midway
                           Research Computing Center
                             University of Chicago
                            http://rcc.uchicago.edu


For the RCC Manual see the documentation site at
    http://docs.rcc.uchicago.edu

To check your allocation balance for the current period use this command:
accounts balance

Questions and issues should be sent to help@rcc.uchicago.edu or 773-795-2667.

*******************************************************************************

Midway information system usage may be monitored, recorded, and subject to audit.
Use of the information system indicates consent 

In [4]:
%close

Close <rcc> Done!


Another way of create a SSH instance is using a dict (Note that multi_factor_auth_option must be string)

In [9]:
config = {'hostname': 'midway2.rcc.uchicago.edu',
          'port': 22,
          'timeout': 30,
          'username': 'your user name',
          'password': 'your password',
          'multi_factor_auth': 'yes',
          'multi_factor_auth_key': 'Duo',
          'multi_factor_auth_option': '1',


          'REMOTE_WORKING_DIR': '/home/your_username/simulation',
          'REMOTE_SCRATCH_DIR': '/home/your_username/scratch-midway2/simulation',


          'log_file': 'log.pickle',


          'cmd_job_status': 'squeue -u your_username',
          'cmd_job_submit': 'sbatch',
          'cmd_job_cancel': 'scancel',

          'job_status_header_skip': 1,
          'job_status_jobid_col': 1,
          'job_status_st_col': 5,
          'job_status_pending_label': 'PD',
          'job_status_running_label': 'R',
          'job_status_time_col': 6}

In [6]:
ssh=SSH(config,'rcc')
ssh.connect()

Password:
Duo two-factor login for huangl1

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4177
 2. Phone call to XXX-XXX-4177
 3. SMS passcodes to XXX-XXX-4177

Passcode or option (1-3):
Success. Logging you in...
Last login: Wed Jul 22 03:47:14 2020 from c-73-51-208-105.hsd1.il.comcast.net
                               Welcome to Midway
                           Research Computing Center
                             University of Chicago
                            http://rcc.uchicago.edu


For the RCC Manual see the documentation site at
    http://docs.rcc.uchicago.edu

To check your allocation balance for the current period use this command:
accounts balance

Questions and issues should be sent to help@rcc.uchicago.edu or 773-795-2667.

*******************************************************************************

Midway information system usage may be monitored, recorded, and subject to audit.
Use of the information system indicates consent 

# Line and Cell Magic

In [3]:
%r ls

ls
[huangl1@midway2-login2 simulation]$ 
 

In [4]:
%%r

ls -a
pwd
ls -l


[huangl1@midway2-login2 simulation]$ ls -a
.  ..  .ezjob.py
[huangl1@midway2-login2 simulation]$ pwd
/home/huangl1/simulation
[huangl1@midway2-login2 simulation]$ ls -l
total 0
[huangl1@midway2-login2 simulation]$ 
 

ssh.shell() is the same as %r %rr

In [5]:
ssh.shell('ls')

ls
[huangl1@midway2-login2 simulation]$ 
 

ssh.cmd() gives the returned value

In [6]:
return_string = ssh.cmd("cd /home/huangl1/scratch-midway2/simulation/0D; ls")
print(return_string)

30_centered_relax.xyz
30_si35h36.xyz
H_ONCV_PBE-1.2.upf
Si_ONCV_PBE-1.2.upf
ecut_test
energies.ecutwfc.H_ONCV_PBE-1.2.upf
energies.ecutwfc.Si_ONCV_PBE-1.2.upf
fig.ecutwfc.H_ONCV_PBE-1.2.upf.png
fig.ecutwfc.Si_ONCV_PBE-1.2.upf.png
in
out.ecut_wfc.H_ONCV_PBE-1.2.upf
out.ecut_wfc.Si_ONCV_PBE-1.2.upf
pw_cutoff.sh
relax.in
relax.out
relax.xyz
run_relax.sh
si35h36.xyz
slurm-2196851.out
slurm-2252713.out
vacuum_test
wfreq_test



If your prefer terminal we have both %ezterm and %ezxterm

In [8]:
%ezterm

# Template

## from template string

use jinja2 template engine, variables are defined as {{var_name}}

In [9]:
template_str="""#!/bin/bash
#SBATCH --job-name={{job_name}}
#SBATCH --time={{time}}
#SBATCH --partition={{queue}}
#SBATCH --nodes={{nodes}}
#SBATCH --ntasks-per-node={{ntasks_per_node}}
#SBATCH --cpus-per-task={{cpus_per_task}}
#SBATCH --mem-per-cpu={{mem_per_cpu}}

module purge
module load intelmpi/5.1+intel-16.0 mkl/2017.up4 Anaconda3/5.1.0

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK
NTASKS=$(($SLURM_NTASKS_PER_NODE * $SLURM_JOB_NUM_NODES))

{{cmd}}
"""

In [10]:
parameters = {'job_name':'pw',
              'time':'00:20:00',
             'queue':'broadwl',
             'nodes':1,
             'ntasks_per_node':8,
             'cpus_per_task':1,
             'mem_per_cpu':2000,
             'cmd':'mpirun -np $NTASKS pw.x -i pw.in > pw.out'}

In [11]:
rcc_template_from_string = Template(template_str,parameters)

## from template file

In [12]:
rcc_template_from_file = Template('../template_rcc.sh',parameters)

## write your rendered template to  disk

In [13]:
#generate the file, if debug=True  print the template
rcc_template_from_string.write('rcc_from_string.sh',debug=True)
rcc_template_from_file.write('rcc_from_file.sh')

#!/bin/bash
#SBATCH --job-name=pw
#SBATCH --time=00:20:00
#SBATCH --partition=broadwl
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=2000

module purge
module load intelmpi/5.1+intel-16.0 mkl/2017.up4 Anaconda3/5.1.0

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK
NTASKS=$(($SLURM_NTASKS_PER_NODE * $SLURM_JOB_NUM_NODES))

mpirun -np $NTASKS pw.x -i pw.in > pw.out


In [14]:
%%bash
ls

EZjob_Tutorial_Basic.ipynb
EZjob_Tutorial_EZterm.ipynb
EZjob_Tutorial_EZxterm.ipynb
EZjob_Tutorial_MultiRemoteServerJobs.ipynb
EZjob_Tutorial_PortForwarding.ipynb
rcc
rcc_from_file.sh
rcc_from_string.sh


## dos2unix 

In [15]:
#In some case your remote server only takes unix format, you can convert your local file by
Template.dos2unix('filename')

filename not found!


In [16]:
#create a testfile
with open('testfile','w') as f:
    f.write("""
    testfile win foramt
    testfile win foramt
    """)

In [17]:
%%bash
cat testfile


    testfile win foramt
    testfile win foramt
    

In [18]:
Template.dos2unix('testfile')

dos2unix:testfile


# JobManager

In [19]:
#JobManager takes two args <ssh> <name>.
#Both of them are optinal
#It can take a SSH instance to init
#If no ssh instance provided and only one valid ssh instance exists.
#The only valid one will be used. If multi valid ssh instances exist. User will choose one.
#<name> is optional, it used to distinguish diffrent JobManager instances when you have multi job managers. 
#JobManger will auto change the dir to LOCAL_WORKING_DIR
jmn=JobManager(ssh)

Change directory to LOCAL_WORKING_DIR:C:\Users\headh\Desktop\0D\tutorial\rcc


In [20]:
#check our current location
%pwd

'C:\\Users\\headh\\Desktop\\0D\\tutorial\\rcc'

In [21]:
#JobManger.view() print current job status
jmn.view()

    JobID  | Status  |   Time   |    SubmitTime       |       Note
 -----------------------------Pending Jobs------------------------------
   2975955 | Pending |     0:00 | 2020-07-14 03:45:52 | Cannot find JobInfo in log 


In [22]:
#Magic method open EZjob watcher, refresh every 10 sec, if not sepecify, default refresh rate is evey 5 secs
%ezjob 10

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

## submit job

You do not need to do ssh.push(), push is auto called when you submit a job, this is making sure the remote files are up to date.

In [23]:
#check current folder at local
%pwd

'C:\\Users\\headh\\Desktop\\0D\\tutorial\\rcc'

In [24]:
#create a folder for simple run
%mkdir simple_run

In [25]:
%cd simple_run

C:\Users\headh\Desktop\0D\tutorial\rcc\simple_run


In [26]:
%pwd

'C:\\Users\\headh\\Desktop\\0D\\tutorial\\rcc\\simple_run'

download QE input file to simple_run folder

In [27]:
%%bash
wget -N -q http://www.west-code.org/doc/training/silane/pw.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.0.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/Si_ONCV_PBE-1.1.upf

write run.sh file for job submission

In [28]:
#use the rcc_template, if debug=True, it will also print on screen
rcc_template_from_file.write('run.sh',debug=True)

#!/bin/bash
#SBATCH --job-name=pw
#SBATCH --time=00:20:00
#SBATCH --partition=broadwl
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=2000

module purge
module load intelmpi/5.1+intel-16.0 mkl/2017.up4 Anaconda3/5.1.0

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK
NTASKS=$(($SLURM_NTASKS_PER_NODE * $SLURM_JOB_NUM_NODES))

mpirun -np $NTASKS pw.x -i pw.in > pw.out



In [29]:
#check current dir
%pwd

'C:\\Users\\headh\\Desktop\\0D\\tutorial\\rcc\\simple_run'

In [30]:
%%bash
ls

H_ONCV_PBE-1.0.upf
Si_ONCV_PBE-1.1.upf
pw.in
run.sh


Now we have all files needed for this simple run at folder rcc/simple_run

In [31]:
#since we direct output to pw.out at template file, we will collect pw.out
#collect is a list of files and folders to collect 
jmn.submit(file='run.sh',collect=['pw.out'])

synchronizing local:C:\Users\headh\Desktop\0D\tutorial\rcc  ->  remote:/home/huangl1/simulation
Upload local  dir:C:\Users\headh\Desktop\0D\tutorial\rcc\simple_run


'3009581'

the above job submit action can also be done using magic function

In [ ]:
%submit run.sh @collect=pw.out

If you want to collect  slurm* log files and copy them to REMOTE_WORKING_DIR after job done 

In [ ]:
%submit run.sh @collect=pw.out,slurm*.log

The magical function:
```
%submit <HPC_bash_file>  <optional:@collect=collect1,collect2,collect3,log*>  <optional:@scratch=/Your/remote/scratch/defined/here/will/overwrite/config/scrach/setting>
```
The jm.submit way:
```
jm = JobManager(ssh_instance)
jm.submit('run.sh',collect=['file1','folder1','prefix*'])
```

- auto push when submitting a job, auto push you local files/folder change to remote.
- auto path translate, local job submit action will auto mirror to the corresponding remote folder, submit at local, run at remote.
- collect = ['result.out','result*'...]  copy result files/folders from remote scratch to remote working after job done. keep remote & local working dir small.
- If you leave out collect, the default value is '*'.  which means you will copy all files/folders generated by simulation from remote scrach back to remote working dir.
- If you do not define REMOTE_SCRATCH_DIR in your config file, the REMOTE_WORKING_DIR will be used as scratch, collect option is ignored.


* You can customize your note, if note is None, note will be the <job submit path|collect>
* You can customize your scratch, this will overwrite default setting is config file of REMOTE_SCRATCH_DIR
```
jmn.submit(file='run.sh',collect=['pw.out'],note='customized_note',scratch='/your/customized/scratch')
```

In [32]:
#view current job status
jmn.view()

    JobID  | Status  |   Time   |    SubmitTime       |       Note
 -----------------------------Pending Jobs------------------------------
   2975955 | Pending |     0:00 | 2020-07-14 03:45:52 | Cannot find JobInfo in log 
   3009581 | Pending |     0:00 | 2020-07-15 21:30:22 | \simple_run | pw.out 


Let's check that the simple_run folder is mirror to remote and run at scratch

In [36]:
%%r
ls
pwd


[huangl1@midway2-login2 simulation]$ ls
simple_run
[huangl1@midway2-login2 simulation]$ pwd
/home/huangl1/simulation
[huangl1@midway2-login2 simulation]$ 
 

we can see simple_run is in /home/huangl1/simulation which is my REMOTE_WORKING_DIR

In [40]:
%%r
cd ~ 
cd scratch-midway2
cd simulation
ls
pwd


[huangl1@midway2-login2 simulation]$ 
 cd ~ 
[huangl1@midway2-login2 ~]$ cd scratch-midway2
[huangl1@midway2-login2 scratch-midway2]$ cd simulation
[huangl1@midway2-login2 simulation]$ ls
0D  log.pickle  simple_run
[huangl1@midway2-login2 simulation]$ pwd
/home/huangl1/scratch-midway2/simulation
[huangl1@midway2-login2 simulation]$ 
 

we can see simple_run is copyed to /home/huangl1/scratch-midway2/simulation which is my REMOTE_SCRATCH_DIR

cancel jobs

In [ ]:
jmn.cancel(2989148)
jmn.cancel(2668449,2668450)

In [41]:
jmn.view()

    JobID  | Status  |   Time   |    SubmitTime       |       Note
 -----------------------------Pending Jobs------------------------------
   2975955 | Pending |     0:00 | 2020-07-14 03:45:52 | Cannot find JobInfo in log 
   3009581 | Pending |     0:00 | 2020-07-15 21:30:22 | \simple_run | pw.out 


You can also cancel job at EZjob Watcher, just click X at left side of JobID

The Note section in EZjob Watcher by default is <path | collect> if you do not provide customized not when do a job submission. 
if the note is <Cannot find JobInfo in log>, this means you may submit your job at remote directly, therefore, ezjob have no history of this job.

In [42]:
#you can close ezjob watcher by 
%ezjob close

In [43]:
#If you want to go to the top folder
jmn.cd_LOCAL_WORKING_DIR()

Change directory to LOCAL_WORKING_DIR:C:\Users\headh\Desktop\0D\tutorial\rcc


# Pull and Push

In [44]:
#currently I am at
%pwd

'C:\\Users\\headh\\Desktop\\0D\\tutorial\\rcc'

In [45]:
#find out our Local working dir
ssh.get_LOCAL_WORKING_DIR()

'C:\\Users\\headh\\Desktop\\0D\\tutorial\\rcc'

In [46]:
#we are working at this dir remotely
ssh.get_REMOTE_WORKING_DIR()

'/home/huangl1/simulation'

In [47]:
#job submission scratch folder remotely is
ssh.get_REMOTE_SCRATCH_DIR()

'/home/huangl1/scratch-midway2/simulation'

## pull

In [48]:
#default delete=Flase, debug=True, with debug=True, will print local file changes after pull
#let's pull simple_run result to local
ssh.pull()

synchronizing remote:/home/huangl1/simulation  ->  local:C:\Users\headh\Desktop\0D\tutorial\rcc
--------------------------- Local files change after pull -------------------------------
------------------------------------------------------------------------------------------


In [49]:
#delete=True will mirror remote to local,
#if you create a file at local but not at remote, 
#the local file or folder will be deleted
ssh.pull(delete=True)

synchronizing remote:/home/huangl1/simulation  ->  local:C:\Users\headh\Desktop\0D\tutorial\rcc
--------------------------- Local files change after pull -------------------------------
------------------------------------------------------------------------------------------


Let's create file/folder at local 

In [50]:
%pwd

'C:\\Users\\headh\\Desktop\\0D\\tutorial\\rcc'

In [51]:
%mkdir test

In [52]:
%%bash 
touch test.txt

In [53]:
%%bash 
ls

simple_run
test
test.txt


In [54]:
#this will delete folder test and file test.txt since they are not at remote 
ssh.pull(delete=True)

synchronizing remote:/home/huangl1/simulation  ->  local:C:\Users\headh\Desktop\0D\tutorial\rcc
Delete  local  dir:C:\Users\headh\Desktop\0D\tutorial\rcc\test
Delete  local  file:C:\Users\headh\Desktop\0D\tutorial\rcc\test.txt
--------------------------- Local files change after pull -------------------------------
Files Deleted:
	C:\Users\headh\Desktop\0D\tutorial\rcc\test.txt
Folder Deleted:
	C:\Users\headh\Desktop\0D\tutorial\rcc\test
------------------------------------------------------------------------------------------


you can also use magic function %pull , this is the same as ssh.pull(delete=False)

In [ ]:
%pull

this is same as ssh.pull(delete=True)

In [ ]:
%pull --delete

If more than one ssh instances exist, one can choose which one to do the pull/push

In [ ]:
%pull @choose

## push

In [55]:
#default delete=False, debug = True, if debug=True, will print local file changes before push
ssh.push()

-----------------------Tracking local files change before push ---------------------------
------------------------------------------------------------------------------------------
synchronizing local:C:\Users\headh\Desktop\0D\tutorial\rcc  ->  remote:/home/huangl1/simulation


Let's create folder and files at local and do a push

In [56]:
%mkdir test

In [57]:
%%bash
touch test.txt

In [58]:
%%bash
ls

simple_run
test
test.txt


In [59]:
#do the push
ssh.push()

-----------------------Tracking local files change before push ---------------------------
Files Added:
	C:\Users\headh\Desktop\0D\tutorial\rcc\test.txt
Folder Added:
	C:\Users\headh\Desktop\0D\tutorial\rcc\test
------------------------------------------------------------------------------------------
synchronizing local:C:\Users\headh\Desktop\0D\tutorial\rcc  ->  remote:/home/huangl1/simulation
Upload local  dir:C:\Users\headh\Desktop\0D\tutorial\rcc\test
Upload local  file:C:\Users\headh\Desktop\0D\tutorial\rcc\test.txt


In [60]:
# if delete=True, any file at remote not at local will be deleted at remote.
# This delete=True force remote look exactly like local.
ssh.push(delete=True)

-----------------------Tracking local files change before push ---------------------------
------------------------------------------------------------------------------------------
synchronizing local:C:\Users\headh\Desktop\0D\tutorial\rcc  ->  remote:/home/huangl1/simulation


lets create files and folder at remote

go back to my remote_working_dir first

In [63]:
%r  cd {ssh.get_REMOTE_WORKING_DIR()}

 cd /home/huangl1/simulation
[huangl1@midway2-login2 simulation]$ 
 

In [64]:
%r ls

ls
simple_run  test  test.txt
[huangl1@midway2-login2 simulation]$ 
 

In [65]:
%%r
mkdir remote_dir
touch remote_file


[huangl1@midway2-login2 simulation]$ mkdir remote_dir
[huangl1@midway2-login2 simulation]$ touch remote_file
[huangl1@midway2-login2 simulation]$ 
 

In [66]:
%r ls

ls
remote_dir  remote_file  simple_run  test  test.txt
[huangl1@midway2-login2 simulation]$ 
 

In [67]:
ssh.push(delete=True)

-----------------------Tracking local files change before push ---------------------------
------------------------------------------------------------------------------------------
synchronizing local:C:\Users\headh\Desktop\0D\tutorial\rcc  ->  remote:/home/huangl1/simulation
Delete remote dir:/home/huangl1/simulation//remote_dir
Delete remote file:/home/huangl1/simulation//remote_file


In [68]:
%r ls

ls
simple_run  test  test.txt
[huangl1@midway2-login2 simulation]$ 
 

another way of push is to use magic function, it is the same as ssh.push(delete=False)

In [ ]:
%push 

This is the same as ssh.push(delete=True)

In [ ]:
%push --delete

# Advanced(optional): Paramiko SFTP object

more on http://docs.paramiko.org/en/stable/api/sftp.html

In [ ]:
sftp = ssh.get_sftp()

In [ ]:
sftp.mkdir("demo_sftp_folder")

with sftp.open("README", "w") as f:
    f.write("This will write to remote file.\n")
    
with open("demo_sftp.py", "r") as f:
    data = f.read()

sftp.open("demo_sftp_folder/demo_sftp.py", "w").write(data)

 
with sftp.open("demo_sftp_folder/README", "r") as f:
        data = f.read()
        
with open("README_demo_sftp", "w") as f:
    f.write(data)
 
 
sftp.put("demo_sftp.py", "demo_sftp_folder/demo_sftp.py")
sftp.get("demo_sftp_folder/README", "README_demo_sftp")

# always close ssh after use

In [70]:
#If you want to close a specific ssh instance
ssh.close()

In [8]:
#this is a magic function which close all valid ssh instance.
%close

Close <rcc> Done!
